In [1]:
import pandas as pd
import numpy as np
import scipy

import math as mt

import pickle as pkl
import os
import psycopg2
import matplotlib
import matplotlib.pyplot as plt
from numpy.random import normal
import calendar
from scipy.optimize import curve_fit

In [2]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,8)
import warnings
warnings.filterwarnings('ignore')

In [3]:
#import plotly
import plotly.plotly as py
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [4]:
# Let's define a function that will simulate an occupancy level (demand)
# for the next "nb_days" days.

def simulate_demand(nb_days, period=7.0, dc_level=35.0, noise_level=5.0,
                    amplitude=3.0, phase=2.0*mt.pi/7.0, trend=0.06):
    """Simulate a demand curve over a forecast period.

    Parameters
    ----------

    nb_days (integer):
       Length of the forecast period (in days)

    period (float):
       Length of the seasonal component. Typically 7 days for weekly
       fluctuations.

    dc_level (float):
       Baseline occupancy level

    noise_level (float):
       Noise level (additive) of the occupancy numbers. This number
       corresponds to the standard deviation of the normal distribution.

    amplitude (float):
       Amplitude of the periodic signal

    phase (float):
       phase shift of the signal (in radians)

    trend (float):
       slope of the linear trend term. In units of rooms/day. Set to 0 if you
       do not want to include a linear trend.

    Returns
    -------
    A numpy.ndarray (1D) containing the forecast numbers.
    """

    days = np.linspace(1,nb_days,nb_days)

    # noise level:
    noise = noise_level * normal(size=len(days))

    # Demand signal:
    demand = dc_level + trend * days + amplitude * \
             np.cos(2.0 * mt.pi * days / period + phase) + noise

    return demand

In [7]:
# Let's simulate the occupancy for a period of 90 days:

date_start = '2017-07-31'
date_end = '2017-10-29'
nb_days = (pd.to_datetime(date_end) - pd.to_datetime(date_start)).days

forecasted_demand = simulate_demand(nb_days, dc_level=68,
                                    noise_level=1.0, amplitude=5)

forecasted_demand = list(map(round, forecasted_demand))

forecasted_demand = pd.DataFrame(forecasted_demand,
                                 index=pd.date_range(start=date_start,
                                                     end=date_end,
                                                     closed='left'),
                                                     columns=['occupancy'])

In [8]:
# Let's plot the occupancy forecast using plotly:

occupancy = [go.Scatter(x=forecasted_demand.index, y=forecasted_demand['occupancy'],
                        name='Forecasted Junior Suites with City View Occupancy for the next 90 days')]

layout_occ = go.Layout(title='Forecasted Junior Suites with City View Occupancy -- Each room is priced $120/night',
                       xaxis={'title':'Day'},
                       yaxis={'title':'Number of Rooms Occupied'},
                       shapes=[{'type':'line',
                                'x0':'2017-07-31',
                                'x1':'2017-10-31',
                                'y0':80.0, 'y1':80.0,
                                'line': {
                                    'color': 'rgb(50, 171, 96)',
                                    'width': 4, 'dash':'dashdot'}
                               }]
                      )

fig = go.Figure(data=occupancy, layout=layout_occ)
iplot(fig, filename='occ_ts')

In [9]:
# Let's assume a demand price elasticity function:

def demand_price_elasticity(price, nominal_demand, elasticity=-2.0, nominal_price=120.0):
    """Returns demand given a value for the elasticity, nominal demand and nominal price.

    Parameters
    ----------

    price (numpy.ndarray):
        one-dimensional price array. The length of that array should correspond to the
        length of the forecast period.

    nominal_demand (numpy.ndarray):
        one-dimensional forecasted occupancy array. The length of that array should
        correspond to the length of the forecast period.

    elasticity (float):
        value of the elasticity between price and demand. A value of e=-2 is reasonable.

    nominal_price (float):
        room rate for which the forecast was computed.

    Returns
    -------

    A numpy.ndarray of expected demand.
    """

    return nominal_demand * ( price / nominal_price ) ** (elasticity)

In [10]:
import scipy.optimize as optimize

In [31]:
# definition of the objective function:

def objective(p_t, nominal_demand=np.array([50,40,30,20]),
              elasticity=-2.0, nominal_price=120.0):
    """
    Definition of the objective function. This is the function that want to minimize.
    (minus sign in front)

    Parameters
    ----------

    p_t (numpy.ndarray):
        one-dimensional price array. The length of that array should correspond to the
        length of the forecast period.

    nominal_demand (numpy.ndarray):
        one-dimensional forecasted occupancy array. The length of that array should
        correspond to the length of the forecast period.

    elasticity (float):
        value of the elasticity between price and demand. A value of e=-2 is
        reasonable.

    nominal price (float):
        room rate for which the forecast was computed.

    Returns
    -------

    Value of the objective function (float).

    Note: here we're trying to minimize the objective function. That's where the
    minus sign comes_in.

    """
    
    return (-1.0 * np.sum( p_t * demand_price_elasticity(p_t, nominal_demand=nominal_demand,
                                                        elasticity=elasticity,
                                                        nominal_price=nominal_price) )) / 100

In [32]:
# Constraints:

def constraint_1(p_t):
    """ This constraint ensures that the prices are positive.
    """
    return p_t


def constraint_2(p_t, capacity=20, forecasted_demand=35.0,
                 elasticity=-2.0, nominal_price=120.0):
    """ This constraint ensures that the demand does not exceed
    capacity.

    Parameters
    ----------

    p_t (float):
        Room price

    capacity (integer):
        Capacity of the hotel (in rooms).

    forecasted_demand (float):
        Forecasted demand (in rooms) for that night

    elasticity (float):
        slope of the

    nominal_price (float):
        The price for which the forecasted_demand was computed.

    Returns
    -------
    Returns an array of excess capacity.

    """
    return capacity - demand_price_elasticity(p_t, nominal_demand=forecasted_demand,
                                                        elasticity=elasticity,
                                                        nominal_price=nominal_price)

In [33]:
# Let's run the optimization algorithm over four overlapping segments
# of 20, 40, 60, 80 room capacity.

# We look at four capacity segments: 20, 40, 60, and 80 (full capacity)
# rooms available.
capacities = [20.0, 40.0, 60.0, 80.0]

optimization_results = {}
for capacity in capacities:

    # Nominal price associated with forecasted demand:
    nominal_price = 120.0
    # Forecasted demand:
    nominal_demand = forecasted_demand['occupancy'].values
    # Assumed price elasticity:
    elasticity = -2.0

    # Starting values:
    p_start = 125.0 * np.ones(len(nominal_demand))

    # bounds on the prices. Let's stick with reasonable values.
    # One could be more sophisticated here and apply constraints
    # that limit the prices to be in range of what competitors
    # are charging, for example.
    bounds = tuple((10.0, 400.0) for p in p_start)

    # Constraints:
    constraints = ({'type': 'ineq', 'fun':  lambda x:  constraint_1(x)},
               {'type': 'ineq', 'fun':  lambda x, capacity=capacity,
                                           forecasted_demand=nominal_demand,
                                           elasticity=elasticity,
                                           nominal_price=nominal_price: constraint_2(x,capacity=capacity,
                                                                                     forecasted_demand=nominal_demand,
                                                                                     elasticity=elasticity,
                                                                                     nominal_price=nominal_price)})

    opt_results = optimize.minimize(objective, p_start, args=(nominal_demand,
                                                              elasticity,
                                                              nominal_price),
                                    method='SLSQP', bounds=bounds,
                                    constraints=constraints)

    optimization_results[capacity] = opt_results

In [14]:
# Plotting the resulting rates vs dates.

time_array = np.linspace(1,len(nominal_demand),len(nominal_demand))
rate_df = pd.DataFrame(index=time_array)

for capacity in optimization_results.keys():
    rate_df = pd.concat([rate_df,
                         pd.DataFrame(optimization_results[capacity]['x'],
                                      columns=['{}'.format(capacity)],
                                      index=time_array)],
                        axis=1)

rate_df.index.name = 'Day'
datelist = pd.date_range(start=date_start, end=date_end, closed='left').tolist()
rate_df.index = [ x.date() for x in datelist]

In [27]:
# Generate a pretty table for display purposes.

rate_df_to_show = rate_df.copy()

# Renaming the columns:
rate_df_to_show = rate_df_to_show[np.sort(np.asarray(rate_df_to_show.columns))]
rate_df_to_show.columns = ['Capacity left : {}'.format(x) for x in rate_df_to_show.columns]

# Rounding the numbers:
for col in rate_df_to_show.columns:
    rate_df_to_show[col] = rate_df_to_show[col].apply(lambda x: round(x,2))

dow_map = { 6:'Sun', 0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat'}
rate_df_to_show['date'] = rate_df_to_show.index
rate_df_to_show['dow'] = rate_df_to_show['date'].apply(lambda x: dow_map[x.weekday()])
rate_df_to_show['date'] = rate_df_to_show.apply(lambda row: row['dow']+" "+str(row['date']),
                                                axis=1)
rate_df_to_show.index = rate_df_to_show['date'].values
rate_df_to_show.drop(['date','dow'],axis=1,inplace=True)
rate_df_to_show.head(20)


,Capacity left : 20.0,Capacity left : 40.0,Capacity left : 60.0,Capacity left : 80.0
Mon 2017-07-31,217.99,154.14,125.86,109.00
Tue 2017-08-01,212.98,150.60,122.96,106.49
Wed 2017-08-02,212.98,150.60,122.96,106.49
Thu 2017-08-03,217.99,154.14,125.86,109.00
Fri 2017-08-04,226.10,159.87,130.54,113.05
Sat 2017-08-05,232.38,164.32,134.16,116.19
Sun 2017-08-06,226.10,159.87,130.54,113.05
Mon 2017-08-07,221.27,156.46,127.75,110.63
Tue 2017-08-08,211.28,149.40,121.98,105.64
Wed 2017-08-09,212.98,150.60,122.96,106.49


In [16]:
# Plotting the room rate time series.
# Let's focus on a single week cycle.

price_levels = [go.Scatter(x=rate_df_to_show.head(7).index,
                           y=rate_df_to_show.head(7)['Capacity left : 20.0'],
                           name='Capacity Remaining : 20 rooms'),
                go.Scatter(x=rate_df_to_show.head(7).index,
                           y=rate_df_to_show.head(7)['Capacity left : 40.0'],
                           name='Capacity Remaining : 40 rooms'),
                go.Scatter(x=rate_df_to_show.head(7).index,
                           y=rate_df_to_show.head(7)['Capacity left : 60.0'],
                           name='Capacity Remaining : 60 rooms'),
                go.Scatter(x=rate_df_to_show.head(7).index,
                           y=rate_df_to_show.head(7)['Capacity left : 80.0'],
                           name='Capacity Remaining : 80 rooms')]

layout_prices = go.Layout(title='Rate vs Reservation Date and Current Capacity Levels',
                       xaxis={'title':'Day'}, yaxis={'title':'Rate ($)'})

fig = go.Figure(data=price_levels, layout=layout_prices)
iplot(fig, filename='price_levels_ts')